<a href="https://colab.research.google.com/github/CRag-01/Crypto-Token-Analysis---Vizualisation/blob/main/ADA_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
# import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly

In [3]:
import warnings

In [4]:
pip install yfinance

     |████████████████████████████████| 6.3 MB 7.6 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [5]:
import yfinance as yf

In [20]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '₹{:,.2f}'.format

In [21]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
eth_df = yf.download('ADA-INR',start_date, today)
eth_df.to_csv("ADA.csv")

[*********************100%***********************]  1 of 1 completed


In [22]:
eth_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-09-18,₹3.51,₹3.91,₹3.48,₹3.79,₹3.79,9841662775
2019-09-19,₹3.79,₹3.90,₹3.56,₹3.75,₹3.75,9235801257
2019-09-20,₹3.75,₹3.74,₹3.60,₹3.73,₹3.73,6873694294
2019-09-21,₹3.73,₹3.80,₹3.66,₹3.70,₹3.70,5836358907
2019-09-22,₹3.70,₹3.70,₹3.41,₹3.50,₹3.50,6134080262


In [23]:
eth_df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-12-03,₹129.02,₹129.61,₹115.34,₹117.24,₹117.24,201479625983
2021-12-04,₹117.38,₹117.39,₹90.77,₹106.88,₹106.88,306649160541
2021-12-05,₹106.87,₹108.07,₹97.08,₹103.70,₹103.70,228360471509
2021-12-06,₹103.78,₹108.49,₹94.84,₹107.46,₹107.46,199188273462
2021-12-07,₹108.03,₹110.64,₹107.44,₹108.88,₹108.88,201055371264


In [24]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [25]:
df = eth_df[["Date", "Open"]]
new_names = {
    "Date": "ds", 
    "Open": "y",
}
df.rename(columns=new_names, inplace=True)

In [26]:
df.tail()

,ds,y
803,2021-12-03,₹129.02
804,2021-12-04,₹117.38
805,2021-12-05,₹106.87
806,2021-12-06,₹103.78
807,2021-12-07,₹108.03


In [27]:
# Visualisation of the price trend
x = df["ds"]
y = df["y"]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time Series Plot of the ADA Price",
)

In [29]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [31]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
1168,2022-12-03
1169,2022-12-04
1170,2022-12-05
1171,2022-12-06
1172,2022-12-07


In [32]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1168,2022-12-03,₹223.03,₹204.60,₹240.73
1169,2022-12-04,₹219.55,₹200.27,₹235.84
1170,2022-12-05,₹216.74,₹198.55,₹232.58
1171,2022-12-06,₹208.99,₹190.81,₹224.89
1172,2022-12-07,₹202.74,₹184.17,₹217.95


In [33]:
plot_plotly(m, forecast)

In [34]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
print(f" PREDICTED Opening Price of Ethereum tomorrow: {forecast[forecast['ds'] == next_day]['yhat'].item()}")

 PREDICTED Opening Price of Ethereum tomorrow: 103.23191878379698


In [35]:
plot_components_plotly(m, forecast)